In [1]:
import pandas as pd
import numpy as np

import dms_variants.binarymap
import dms_variants.globalepistasis

In [33]:
import pickle
def from_pickle_file(path):
    with open(path, "rb") as file:
        return pickle.load(file)

[aa_func_scores, wtseq] = from_pickle_file('../data/gb1_fitness.pkl')
[aa_stab_scores, wtseq] = from_pickle_file('../data/gb1_stability.pkl')

In [36]:
aa_func_scores.head()

,aa_substitutions,stability_score_sd,stability_score,n_aa_substitutions
0,A20D,0.16780,-1.646,1
1,A20E,0.04353,-0.582,1
2,A20F,0.09556,-1.732,1
3,A20G,0.17730,-1.107,1
4,A20H,0.14330,-1.914,1


In [37]:
aa_stab_scores.head()

,aa_substitutions,stability_score_sd,stability_score,n_aa_substitutions
0,A20D,0.16780,-1.646,1
1,A20E,0.04353,-0.582,1
2,A20F,0.09556,-1.732,1
3,A20G,0.17730,-1.107,1
4,A20H,0.14330,-1.914,1


change 'T2X' to 'Q2X' for gb1 stability scores, to make sure the wt seq lines up for both datasets

In [38]:
aa_stab_scores.drop(['n_aa_substitutions'], axis=1, inplace=True)

import re

single_digit = re.compile('(T)(2)([A-Z])')

aa_stab_scores['aa_substitutions'] = aa_stab_scores['aa_substitutions'].replace(
    {single_digit: r'Q\2\3'}, regex=True)

In [39]:
gb1_full = pd.merge(aa_func_scores,
                    aa_stab_scores,
                    how='outer',
                    on=['aa_substitutions']
                   )

In [40]:
gb1_full.info()
gb1_full.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536980 entries, 0 to 536979
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   aa_substitutions    536980 non-null  object 
 1   input_count         536962 non-null  float64
 2   sele_count          536962 non-null  float64
 3   func_score          536962 non-null  float64
 4   stability_score_sd  935 non-null     float64
 5   stability_score     935 non-null     float64
dtypes: float64(5), object(1)
memory usage: 28.7+ MB


,aa_substitutions,input_count,sele_count,func_score,stability_score_sd,stability_score
0,Q2A,14663.0,38476.0,1.517930,0.08169,-0.4704
1,Q2C,13001.0,23023.0,1.024400,NaN,NaN
2,Q2D,11488.0,18085.0,0.910665,0.01713,-0.5538
3,Q2E,9501.0,15629.0,0.951582,0.03447,0.1299
4,Q2F,4770.0,13332.0,1.616819,0.07045,0.3008


make sure we got func_score and stab_score to line up for overlapping aa_substitutions

In [41]:
gb1_doubles = gb1_full.dropna()
gb1_doubles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 917 entries, 0 to 1044
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   aa_substitutions    917 non-null    object 
 1   input_count         917 non-null    float64
 2   sele_count          917 non-null    float64
 3   func_score          917 non-null    float64
 4   stability_score_sd  917 non-null    float64
 5   stability_score     917 non-null    float64
dtypes: float64(5), object(1)
memory usage: 50.1+ KB


In [42]:
wtseq = "MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE"

In [43]:
# export fitness data and wtseq
import pickle
with open("gb1_full.pkl", "wb") as f:
      pickle.dump([gb1_full, wtseq], f)